In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb

import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

train_loader, val_loader, test_loader = eee_get_dataloaders(args)

# # get input feature  size
# dataiter = next(iter(train_loader))
# batch =  dataiter


# # Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(1, 120, 1024) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)
# repeat the output to match sequence length
dummy_output = dummy_output.repeat(1, 120, 1)

print(dummy_output.shape)




********** ========== **********
Loading dataloader for Segmentation task
Splitting data to windows
Total windowed clips: 1366
Splitting data to windows
Total windowed clips: 398
Splitting data to windows
Total windowed clips: 735
train dataset size:  1366
val dataset size:  398
test dataset size:  735


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 16])
torch.Size([1, 120, 16])


In [2]:
keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)
print(out_classes)

35


In [4]:
modality = args.dataloader_params['modality']
print("Modality: ", modality)

# Find feature dimension
input, feature_size, label = preprocess(next(iter(train_loader)), args.dataloader_params['modality'], model, device=device)
print("Feature size: ", feature_size)

# print(input[:,0,-3:])

args.transformer_params['input_dim'] = feature_size
args.transformer_params['output_dim'] = out_classes

model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

wandb_logger = wandb.init(
    # set the wandb project where this run will be logged
    project="EgoExoEMS",
    group="Keystep Recognition",
    mode="disabled",
    name="Testing on EgoExoEMS with I3D RGB Features - ICRA Model",
    notes="initial attempt ICRA model with I3D RGB features",
    config={
    "args": args,
    }
)



Modality:  ['resnet']
Feature size:  2048


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [16]:
# # # print one batch

for i, batch in enumerate(train_loader):
    input,feature_size, label = preprocess(batch, args.dataloader_params['modality'], model, device=device)
    print("Model input:",input.shape)
    # output = model(input)
    
    # # print("Model output:",output.shape)
    # # print("Label:",label.shape)
    # # Get the majority label along the sequence length dimension
    # majority_label, _ = torch.mode(label, dim=1)  # [batch_size], mode returns (values, indices)
    
    # # print("Majority label:", majority_label.shape)
    
    # print("Model prediction:",output.argmax(dim=1))
    # print("Ground truth:",majority_label.item())   
    # # Adjust output shape if needed to match the label shape before computing loss
    # # Assuming you want to compute the loss between majority_label and pred, no need to repeat output now
    # loss = criterion(output, majority_label)  # Use majority label for loss computation
    
    # print("Loss:",loss.item())
    # break

# results_dir = './results'
# # # # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger, modality=modality)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger, modality=modality)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")
#     results = test_model(model, test_loader, criterion, device, wandb_logger, modality=modality, epoch=epoch, results_dir=results_dir)
#     print(results)
#     break

# dummy_input = torch.randn(1, 3, 1024) # batch, num_frames, channels, height, width
# dummy_input = dummy_input.to(args.device)
# pred = model(dummy_input)
# print(pred.shape)


Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])
Model input: torch.Size([1, 120, 2048])


KeyboardInterrupt: 

In [5]:
import torch
import torch.nn as nn

class CNN_Encoder(nn.Module):
    def __init__(self, feature_dim, out_channels, kernel_size=3):
        super(CNN_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=feature_dim, out_channels=512, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # No change in temporal dimension
            nn.Conv1d(in_channels=512, out_channels=256, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # No change in temporal dimension
            nn.Conv1d(in_channels=256, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)  # No change in temporal dimension
        )

    def forward(self, x):
        # x: [batch, seq_len, feature_dim] -> need to permute for Conv1d
        x = x.permute(0, 2, 1)  # [batch, feature_dim, seq_len]

        x = self.encoder(x)
        # x = torch.mean(x, dim=-1)  # Global average pooling
        return x

feature_size = 1024
hidden_dim = 256
cnn_encoder = CNN_Encoder(feature_size, hidden_dim)

# Test the model
dummy_input = torch.randn(1, 30, 1024)  # batch, seq_len, feature_dim

pred = cnn_encoder(dummy_input)

print(pred.shape)  # [batch, 1]




torch.Size([1, 256, 3])
